# Topic modelling of news headlines for prediction of news category with 8 topics

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier and assume 8 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import wordcloud

import pyLDAvis.lda_model

## Load train set

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

In [6]:
# append words to stopwords that contain no information
stop_words += ['wegen', 'melden', 'meldet', 'können', 'könnte', 'könnten', 'update', 'neu', 'neue', 'neues', 'ohne', 'letzte', 'letzter',
                  'letztes', 'eins', 'zwei', 'drei', 'vier', 'fünf', 'sechs', 'sieben', 'acht', 'neun', 'zehn',
                   'gehen', 'geht', 'wollen', 'wollte', 'wollt', 'jahr', 'jahre', 'fordern', 'fordert',
                  'warnen', 'warnt', 'frühjahr', 'frühling', 'sommer', 'herbst', 'winter', 'erneut', 'deutlich', 'schwer', 'jahren', 'woche',
                  'wochen', 'monat', 'monate', 'tag', 'tage', 'stunden', 'stunde', 'minuten', 'minuten', 'ende', 'beenden', 'endet']

## Extract features from 'title' 

In [7]:
train[train['title_cleaned'].isnull()]

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned
39381,2023-02-16 15:30:15,Champions League,Liveticker und Highlightvideos,Missing,NaN,ZDF heute,Champions League - Liveticker und Highlightvid...,NaN,Champion Liveticker Highlightvideos
66356,2023-09-21 00:30:05,Champions League,Die wichtigsten Spiele im Video.,Missing,NaN,ZDF heute,Champions League - Die wichtigsten Spiele im V...,NaN,wichtig Spiel Video


In [8]:
# TODO workaround for missing title_cleaned -> TODO Solve this in NLP preprocessing!
train['title_cleaned'] = train['title_cleaned'].fillna(train['title'].apply(lambda x: x.lower()))

In [9]:
train[train['title_cleaned'].isnull()]

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)

In [12]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_cleaned'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [13]:
dtm

<67513x21412 sparse matrix of type '<class 'numpy.int64'>'
	with 330205 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.decomposition import LatentDirichletAllocation

In [15]:
lda = LatentDirichletAllocation(n_components=8, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=8, random_state=42)

## Analyse extracted features

In [16]:
len(cv.get_feature_names_out())

21412

In [17]:
cv.get_feature_names_out()

array(['000', '034', '039', ..., 'войны', 'российские', 'украинцы'],
      dtype=object)

In [18]:
len(lda.components_)

8

In [19]:
lda.components_

array([[ 0.12508604, 13.57035003,  0.75527885, ...,  0.12500003,
         0.12500003,  0.12500006],
       [ 1.73063336,  4.6633913 ,  0.12510378, ...,  2.1249997 ,
         2.1249997 ,  0.12500008],
       [ 0.12506378,  8.2072425 , 26.97819673, ...,  0.12500004,
         0.12500004,  0.12500008],
       ...,
       [ 6.79546959,  5.02608888,  0.1250374 , ...,  0.12500004,
         0.12500004,  0.12500009],
       [ 0.12510076, 13.46324277,  0.12506088, ...,  0.12500005,
         0.12500005,  2.12499941],
       [ 7.84854962,  0.12505316,  0.1250291 , ...,  0.12500005,
         0.12500005,  0.1250001 ]])

In [20]:
len(lda.components_[0])

21412

### Show most important words of first extracted topic

In [21]:
first_topic = lda.components_[0]

In [22]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([20969, 16516,   413, ..., 15091, 10150, 18325], dtype=int64)

In [23]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [24]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

liveblog
selenskyj
angriff
kiew
lage
putin
russisch
russland
krieg
ukraine


### Show most important words of first extracted topic

In [25]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['moskau', 'stimme', 'invasion', 'ukrainisch', 'nato', 'liveblog', 'selenskyj', 'angriff', 'kiew', 'lage', 'putin', 'russisch', 'russland', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #1
['land', 'generation', 'panzer', 'faeser', 'eu', 'grüne', 'fdp', 'streit', 'lindner', 'habeck', 'regierung', 'cdu', 'spd', 'ampel', 'euro']


Die TOP-15 Wörter für das Thema #2
['ulrich', 'habeck', 'groß', 'besuch', 'eu', 'million', 'usa', 'taiwan', 'biden', 'us', 'analyse', 'deutschland', 'berlin', 'scholz', 'china']


Die TOP-15 Wörter für das Thema #3
['verurteilen', 'deutschland', 'tod', 'podcast', 'italien', 'partei', 'groß', 'haft', 'klima', 'putin', 'russland', 'ex', 'israel', 'afd', 'deutsch']


Die TOP-15 Wörter für das Thema #4
['mindestens', 'kommentar', 'mensch', 'mann', 'retten', 'gabor', 'steingart', 'europa', 'iran', 'sanktion', 'gas', 'gastbeitrag', 'frau', 'tote', 'eu']


Die TOP-15 Wörter für das Thema #5
['dax', 'steigen', 'präsident', 'de

### Get topics with highest probability for news items in train data

In [26]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 8)

In [27]:
topic_results[0].round(5)

array([0.01563, 0.40514, 0.01564, 0.34961, 0.01564, 0.16708, 0.01563,
       0.01563])

In [28]:
topic_results[0].argmax()

1

In [29]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [30]:
train.head()

,date,title,description,category,text,source,title_description_text,title_cleaned,title_description_text_cleaned,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,Preis grüne halten Senkung Spritsteuer falsch ...,Preis grüne halten Senkung Spritsteuer falsch ...,1
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,Biden warnen Putin USA liefern modern Raketens...,Biden warnen Putin USA liefern modern Raketens...,0
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,sozial Medium FDP-Politiker Kuhle Internet-Str...,sozial Medium FDP-Politiker Kuhle Internet-Str...,1
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Liveblog ukrainisch Kind vertreiben,Liveblog ukrainisch Kind vertreiben rund Mädch...,0
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,Finanz Dänemark stimmen EU-Verteidigungsvorbehalt,Finanz Dänemark stimmen EU-Verteidigungsvorbeh...,1


## Visualize model

In [31]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      23.551519 -48.784714       1        1  16.058261
1     -19.987120  65.003815       2        1  13.792639
2     -69.285057 -18.713701       3        1  12.737328
5     -11.296581   3.444011       4        1  12.077863
3     -29.980885 -55.602993       5        1  11.660525
6      31.477129  48.689629       6        1  11.364252
4      50.357086  -1.836971       7        1  11.360606
7     -64.923813  35.094860       8        1  10.948526, topic_info=              Term         Freq        Total Category  logprob  loglift
18325      ukraine  4510.000000  4510.000000  Default  30.0000  30.0000
10150        krieg  2613.000000  2613.000000  Default  29.0000  29.0000
15087     russisch  1373.000000  1373.000000  Default  28.0000  28.0000
3583        corona  1141.000000  1141.000000  Default  27.0000  27.0000
15091     russland  1754.000000  1754.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
17168       streik   107.493262   239.467215   Topic8  -5.8248   1.4110
2314        berlin   157.661837   987.479932   Topic8  -5.4417   0.3773
3899   deutschland   161.838060  1672.013111   Topic8  -5.4156  -0.1232
6157          frau   121.035493   565.757718   Topic8  -5.7061   0.6699
17970          tot    99.116654   307.937745   Topic8  -5.9059   1.0784

[490 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
8          5  0.983101            11km
63         7  0.992005              49
138        3  0.972920             abe
418        2  0.264979             afd
418        3  0.101518             afd
...      ...       ...             ...
21131      5  0.169227         zweiter
21131      8  0.724502         zweiter
21182      7  0.978151         ägypten
21275      1  0.991817       überblick
21368      7  0.987032  überschwemmung

[1000 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 6, 4, 7, 5, 8])

## Save train set with topics and dump model

In [32]:
#Save result to csv
train.to_csv('evaluation/train_lda_topics_8.csv')

In [33]:
# Dump countvectorizer
joblib.dump(cv, 'models/cv.jl')

['models/cv.jl']

In [34]:
# Dump LDA model
joblib.dump(lda, 'models/lda_model_8_topics.jl')

['models/lda_model_8_topics.jl']